In [1]:
import time
import torch
import numpy as np
import glob

import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from dataprocess import AugmentColor, ToTensor, ISICDataset
from loss import ISICLoss
from model import LinkNet

import matplotlib.pyplot as plt

import random

In [11]:
batch = 16  # 32 if you have 16 GB of VRAM
batch_per_epoch = 10000 // batch # batch per epoch
model_path = './models/resnet18_001.pth'  # Name for the model save
load_model_path = None  # Load pretrain path
encoder = 'resnet18'  # Encoder type: 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'
final = 'sigmoid'  # Output layer type. 'softmax' or 'sigmoid'

# Image augmentation parameters
gamma = 0.15#0.35
brightness = 1.0#2.0
colors = 0.10#0.25

# ISIC image normalization constants
mean = [0.486, 0.336, 0.275]
std = [0.299, 0.234, 0.209]

dataset_dir = "../input2/data_segmentation_augmented/"
train_dir = dataset_dir + "train"
val_dir = dataset_dir + "val"

In [12]:
np.random.seed(123)

train_transform = transforms.Compose([
    ToTensor(),
    AugmentColor(gamma, brightness, colors),
    transforms.Normalize(mean=mean, std=std)
])

val_transform = transforms.Compose([
    ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

mask_transform = transforms.Compose([
    ToTensor(),
])


train_image_paths = sorted(glob.glob(train_dir + "/img/*.jpeg"))[0:32]
train_target_paths = sorted(glob.glob(train_dir + "/msk/*.jpeg"))[0:32]
train_dataset = ISICDataset(train_image_paths,
                            train_target_paths,
                            img_transform=train_transform,
                            trg_transform=mask_transform)

val_image_paths = sorted(glob.glob(val_dir + "/img/*.jpg"))[0:32]
val_target_paths = sorted(glob.glob(val_dir + "/msk/*.jpg"))[0:32]
val_dataset = ISICDataset(val_image_paths,
                          val_target_paths,
                          img_transform=val_transform,
                          trg_transform=mask_transform)

print("Train imgs:", train_dataset.__len__())
print("Val imgs:", val_dataset.__len__())


Train imgs: 32
Val imgs: 32


In [13]:
def plot_image_mask(img, msk):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(img.permute(1,2,0).cpu().numpy())
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(msk.permute(1,2,0).squeeze().cpu().numpy())


In [ ]:
N = train_dataset.__len__()
img, msk = train_dataset.__getitem__(random.randint(0,N))
print(img.size())
plot_image_mask(img, msk)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


torch.Size([3, 224, 224])


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)

device = torch.device("cuda")

train_loss = ISICLoss().to(device)
val_loss = ISICLoss().to(device)

model = LinkNet(1, 3, encoder, final).to(device)
if load_model_path is not None:
    state = torch.load(load_model_path)
    model.load_state_dict(state)

In [7]:
def val():
    c_loss = 0
    with torch.no_grad():
        for img, trg in val_loader:
            img = img.type(torch.cuda.FloatTensor)
            trg = trg.type(torch.cuda.FloatTensor)
            pred = model(img)
            loss = val_loss(pred, trg)
            c_loss += loss.item()
        c_loss /= val_dataset.__len__()
    return c_loss


In [8]:
def train(epochs):
    losses = []
    best_loss = val()
    print("Start val loss:", best_loss)
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        s_time = time.time()
        for i, (img, trg) in enumerate(train_loader):
            if i > batch_per_epoch:
                break
            # get the inputs
            img = img.type(torch.cuda.FloatTensor)
            trg = trg.type(torch.cuda.FloatTensor)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            pred = model(img) 
            loss = train_loss(pred, trg)
            loss.backward()            
            optimizer.step()
            running_loss += loss.item()
        running_loss /= train_dataset.__len__()
        val_s = val()
        val_s_f = round((2-val_s*batch)/2, 5)  # LB score on val
        print("Epoch:", epoch+1, "train loss:", round(running_loss, 5),
              "val loss", round(val_s, 5), "val score:", val_s_f,
              "time:", round(time.time()-s_time, 2), "s")
        if val_s < best_loss:
            torch.save(model.state_dict(), model_path[:-4] + '_cpt_' \
                       + str(val_s_f) + model_path[-4:])
            best_loss = val_s
            print("Checkpoint saved")
        losses.append([running_loss, val])
        running_loss = 0.0
    # Save the train result
    torch.save(model.state_dict(), model_path[:-4] + '_res_'\
               + str(val_s_f) + model_path[-4:])
    #print(losses)



In [9]:
torch.cuda.synchronize()

# Define the train protocol here
lr = 3e-4
optimizer = optim.Adam(model.parameters(), lr=lr)
train(epochs=100)



NameError: name 'bpe' is not defined

In [ ]:
lr = 1e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
train(epochs=20, batch_per_epoch=bpe, optimizer=optimizer)
lr = 1e-6
optimizer = optim.Adam(model.parameters(), lr=lr)
train(epochs=10, batch_per_epoch=bpe, optimizer=optimizer)


# Save the final result
torch.save(model.state_dict(), model_path)
print('Finished Training')